# Réseaux convolutionnels : CNN

## Keras et Tensorflow

Pour installer Keras et Tensorflow sans GPU :

Pour installer la version GPU sous windows, cf https://medium.com/@raza.shahzad/setting-up-tensorflow-gpu-keras-in-conda-on-windows-10-75d4fd498198  
Sous Linux : http://deeplearning.lipingyang.org/2017/08/01/install-keras-with-tensorflow-backend/  
Sous MacOS (avec GPU Nvidia) : https://blog.wenhaolee.com/run-keras-on-mac-os-with-gpu/

## Initialisations

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

In [ ]:
from keras.datasets import mnist

from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

In [ ]:
# Permet d'éviter les erreurs mémoires pour le GPU
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## MNIST : modèle CNN simple

In [ ]:
digits = datasets.load_digits()
X = digits.images
y = digits.target

In [ ]:
plt.imshow(X[0], cmap="gray_r")

Pour utiliser Tensorflow, on doit reformatter l'image sous la forme **8x8x1** au lieu de **8x8** (pour ajouter le "canal", ici **1** puisque l'image est en niveaux de gris)

In [ ]:
print(X[0][0])     # On affiche la première ligne de la première image avant redimensionnement
X = X.reshape(len(X), 8, 8, 1)
print(X[0][0])     # On affiche la première ligne de la première image après redimensionnement

On normalise chaque pixel entre 0 et 1 (les niveaux sont entre 0 et 16 dans la version sklearn du dataset) :

In [ ]:
# Normalisation entre 0 et 1
X = X.astype('float32') / 16
print(X[0][0])

Comme l'activation d'un neurone classe entre 0 ou 1, on code la cible (classes entre 0 et 9) sous la forme d'un vecteur de 0 ou 1 (*one hot encoding*) avec *to_categorical* :

In [ ]:
print(y[0])
y = to_categorical(y)
print(y[0])

In [ ]:
num_classes = y.shape[1]
print(num_classes)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# Réseau dense simple
model = Sequential()
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


On peut afficher la structure du modèle :

In [ ]:
# Apprentissage
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [ ]:
model.summary()

On va utiliser utiliser une couche convolutionnelle pour l'extraction des caractéristiques, et une couche dense pour la classification :

In [ ]:
# Réseau convolutionnel simple
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(8, 8, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


On peut afficher la structure du modèle :

In [ ]:
model.summary()

In [ ]:
# Apprentissage
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [ ]:
# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

La variable *train* mémorise l'historique des scores sur l'ensemble d'apprentissage :

In [ ]:
print(train.history['accuracy'])

et sur l'ensemble de validation :

In [ ]:
print(train.history['val_accuracy'])

On définit une fonction pour afficher un graphique des scores :

In [ ]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

In [ ]:
# Prediction
y_cnn = model.predict_classes(X_test)

On obtient un vecteur de probabilités :

In [ ]:
print(y_cnn[0])

L'indice de la plus grande probabilité donne la classe prédite :

In [ ]:
print(y_cnn[0].argmax(axis=-1))

In [ ]:
y_cnn_classe = y_cnn.argmax(axis=-1)
y_test_classe = y_test.argmax(axis=-1)

On peut afficher la matrice de confusion :

In [ ]:
cm = confusion_matrix(y_cnn_classe,y_test_classe)
print(cm)
plt.figure(figsize = (12,10))
sns.heatmap(cm, annot=True, cmap="coolwarm")

In [ ]:
import random

plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
for i in range(1,50) :
    ir = random.randint(0,n_test)
    plt.subplot(10,5,i)
    plt.axis('off')
    plt.imshow(X_test[ir].reshape(28,28), cmap="gray_r")
    pred_classe = y_cnn[ir].argmax(axis=-1)
    plt.title('%d / %d' % (y_cnn_classe[ir], y_test_classe[ir]))

## Modèle CNN plus profond

Les images originales MNIST sont au format **28x28** (elles ont été redimensionnées pour le dataset de sklearn, afin de permettre une analyse plus facile avec les approches usuelles de machine learning). Keras dispose du dataset original :

In [ ]:
# Importation des données MNIST de Keras
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
plt.imshow(X_test[0], cmap="gray_r")

In [ ]:
# Redimensionnement des images [echantillon][largeur][hauteur][canaux]
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)

# Normalisation entre 0 et 1
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Encodage des sorties en catégories
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]

On teste un modèle avec deux couches convolutionnelles :

In [ ]:
# Modèle CNN plus profond
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

L'apprentissage peut être un peu long sans GPU ...

In [ ]:
# Apprentissage
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=1)

# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [ ]:
plot_scores(train)

Le modèle entrainé peut être sauvegardé :

In [ ]:
model.save('mnist_cnn2.h5')

On peut ensuite utiliser le modèle sans recommencer l'entraînement :

In [ ]:
new_model = load_model('mnist_cnn2.h5')
new_model.summary()

In [ ]:
scores = new_model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))